In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

from tqdm import tqdm
from numpy import asarray
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# Create Sampling Layer

In [ ]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Encoder

In [ ]:
latent_dim = 2
total_epoch = 100
batch_size = 64
img_dim = (256, 256)

encoder_inputs = keras.Input(shape=(*img_dim, 3))

x = layers.Conv2D(16, kernel_size=(5,5), strides=(2,2), padding='SAME')(encoder_inputs)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2D(32, kernel_size=(5,5), strides=(2,2), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2D(64, kernel_size=(5,5), strides=(2,2), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2D(128, kernel_size=(5,5), strides=(2,2), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)

z_mean    = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z         = Sampling()([z_mean, z_log_var])

encoder = keras.Model(encoder_inputs, 
                      [z_mean, z_log_var, z], name="encoder")

encoder.summary()

## Encoder Plot

In [ ]:
plot_model(encoder, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# Decoder

In [ ]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(16 * 16 * 128, activation="relu")(latent_inputs)
x = layers.Reshape((16, 16, 128))(x)

x = layers.Conv2DTranspose(128, kernel_size=(5,5), strides=(2,2), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(64, kernel_size=(5,5), strides=(2,2), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(32, kernel_size=(5,5), strides=(2,2), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

x = layers.Conv2DTranspose(16, kernel_size=(5,5), strides=(2,2), padding='SAME')(x)
x = layers.BatchNormalization()(x)
x = layers.LeakyReLU(0.2)(x)

decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

## Decoder Plots

In [ ]:
plot_model(decoder, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# Modeling

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            reconstruction_loss = tf.reduce_mean(
                keras.losses.binary_crossentropy(data, reconstruction)
            )
            reconstruction_loss *= 28 * 28
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            "loss": total_loss,
            "reconstruction_loss": reconstruction_loss,
            "kl_loss": kl_loss,
        }

## Training Data

In [ ]:
trn_images = os.listdir('../input/gan-getting-started/monet_jpg')
trn_sizes = []

for i, img_path in enumerate(tqdm(trn_images)):
    img = load_img(os.path.join('../input/gan-getting-started/monet_jpg',
                                f'{img_path}'), target_size=(256, 256))
    img_ary = img_to_array(img)
    trn_sizes.append(img_ary.astype("float32")/255.0)
    
trn_sizes = asarray(trn_sizes)

# Training

In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(trn_sizes, 
        epochs=100, 
        batch_size=batch_size)

## Work In Progress